In [ ]:
# Dependecies
!pip install datasets
#!pip install tensorflow
!pip install pandas-profiling
!pip3 install sweetviz
from datasets import load_dataset
# import tensorflow as tf
# import tensorflow_datasets as tfds
import pandas as pd
import numpy as np

# SBIC EDA

[Social bias frames](https://huggingface.co/datasets/social_bias_frames]) This is the challenge dataset that features microaggressions. "Social Bias Inference Corpus (SBIC) which contains about 150k structured annotations of social media posts, covering over 34k implications about a thousand demographic groups." created by [Washington CS program researchers](https://homes.cs.washington.edu/~msap/social-bias-frames/#:~:text=What%20are%20Social%20Bias%20Frames,standards%20to%20hire%20more%20women.%22). Its mostly reddit, but has posts from Reddit, Gab, and Stormfront posts. 

Annotators: The annotators are Amazon Mechanical Turk workers aged 36±10 years old. The annotators consisted of 55% women, 42% men, and <1% non-binary and 82% identified as White, 4% Asian, 4% Hispanic, 4% Black.

For our model evaluations we will use 



*   post: a string containing the text of the post that was annotated
*   offensiveYN: a string indicating if the post could be offensive to anyone. This is a categorical variable with three possible answers, ‘1.0’ if yes, ‘0.5’ if maybe, ‘0.0’ if no.

Data Splits
To ensure that no post appeared in multiple splits, the curators defined a training instance as the post and its three sets of annotations. They then split the dataset into train, validation, and test sets (75%/12.5%/12.5%).

| name | train | validation |	test | combined
| --- | --- | --- | --- | --- |
| default	| 112900 | 16738	| 17501 | 147139 | 



In [ ]:
sbic_train = load_dataset('social_bias_frames', split='train')
sbic_val = load_dataset('social_bias_frames', split='validation')

Using custom data configuration default


Generating test split:   0%|          | 0/17501 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/16738 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/112900 [00:00<?, ? examples/s]

Dataset social_bias_frames downloaded and prepared to /root/.cache/huggingface/datasets/social_bias_frames/default/0.0.0/79706db13a32c7f9614b997cc4326cbda14e6d3968892a3f0e76c4a970e7e510. Subsequent calls will reuse this data.


Using custom data configuration default
Reusing dataset social_bias_frames (/root/.cache/huggingface/datasets/social_bias_frames/default/0.0.0/79706db13a32c7f9614b997cc4326cbda14e6d3968892a3f0e76c4a970e7e510)


In [ ]:
import sweetviz as sv
df_sbic_train = sbic_train.to_pandas()
df_sbic_val = sbic_val.to_pandas()

In [ ]:
my_report = sv.compare([df_sbic_train, "Training Data"], [df_sbic_val, "Validation Data"])
my_report.show_html("report_sbic_train_vs_val.html")

                                             |          | [  0%]   00:00 -> (? left)

Report report_sbic_train_vs_val.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


# Civil Comments and Covert Toxicity EDA

Data is taken from Civil Comments platform, a commenting plugin for independent news sites. Public comments were created from 2015 - 2017 and appeared on approximately 50 English-language news sites across the world. When Civil Comments shut down in 2017, they chose to make the public comments available. 

https://medium.com/jigsaw/creating-labeled-datasets-and-exploring-the-role-of-human-raters-56367b6db298

The original data, published on figshare, includes the public comment text, some associated metadata such as article IDs, timestamps and commenter-generated "civility" labels, but does not include user ids. Jigsaw extended this dataset by adding additional labels for toxicity, identity mentions, as well as covert offensiveness. This data set is an exact replica of the data released for the [Jigsaw Unintended Bias in Toxicity Classification Kaggle challenge](https://www.kaggle.com/c/jigsaw-unintended-bias-in-toxicity-classification/data). 

For our model training on Civil Comments we will use the features "text" and "toxicity". It looks like the validation and test sets are 5% each of the total number of examples.  

| Split |	Examples
| --- | --- |
| Train |	1,804,874 |
| Validation |	97,320 |
| Test	| 97,320 |

The CivilCommentsCovert set is a subset of CivilCommentsIdentities with ~20% of the train and test splits further annotated for covert offensiveness, in addition to the toxicity and identity labels. Raters were asked to categorize comments as one of explicitly, implicitly, not, or not sure if offensive, as well as whether it contained different types of covert offensiveness.

**Covert toxicity does not have a validation split**

We will probably use 

| Split	| Examples
| --- | --- |
| Train	| 48,074
| Test |	2,455



https://www.kaggle.com/competitions/jigsaw-unintended-bias-in-toxicity-classification/data

test_public_expanded.csv - The public leaderboard test set, including toxicity labels and subgroups. The competition target was a binarized version of the toxicity column, which can be easily reconstructed using a >=0.5 threshold.

test_private_expanded.csv - The private leaderboard test set, including toxicity labels and subgroups. The competition target was a binarized version of the toxicity column, which can be easily reconstructed using a >=0.5 threshold.

In [ ]:
df_civil_full = pd.read_csv('/content/civil_comments.csv', low_memory=False)
len(df_civil_full) - 2*97320  # This is the full dataset with test, validation, and train all combined

1804874

In [ ]:
df_civil_full.columns


Index(['id', 'comment_text', 'created_date', 'publication_id', 'parent_id',
       'article_id', 'rating', 'funny', 'wow', 'sad', 'likes', 'disagree',
       'toxicity', 'severe_toxicity', 'obscene', 'sexual_explicit',
       'identity_attack', 'insult', 'threat', 'identity_annotator_count',
       'toxicity_annotator_count', 'male', 'female', 'transgender',
       'other_gender', 'heterosexual', 'homosexual_gay_or_lesbian', 'bisexual',
       'other_sexual_orientation', 'christian', 'jewish', 'muslim', 'hindu',
       'buddhist', 'atheist', 'other_religion', 'black', 'white', 'asian',
       'latino', 'other_race_or_ethnicity', 'physical_disability',
       'intellectual_or_learning_disability', 'psychiatric_or_mental_illness',
       'other_disability', 'explicitly_offensive', 'implicitly_offensive',
       'not_sure_offensive', 'not_offensive', 'covert_humor',
       'covert_obfuscation', 'covert_emoticons_emojis', 'covert_sarcasm',
       'covert_microaggression', 'covert_masked_ha

In [ ]:
sum(df_civil_full['split'].isna())

0

In [ ]:
df_civil_full['split'].unique()

array(['test_private', 'test_public', 'train'], dtype=object)

test_public_expanded.csv - The public leaderboard test set, including toxicity labels and subgroups. The competition target was a binarized version of the toxicity column, which can be easily reconstructed using a >=0.5 threshold.

test_private_expanded.csv - The private leaderboard test set, including toxicity labels and subgroups. The competition target was a binarized version of the toxicity column, which can be easily reconstructed using a >=0.5 threshold.

So we can use the split values since none are missing. We'll take public as validation and private as test

IDENTITY_LABELS = [
    'male', 'female', 'transgender', 'other_gender', 'heterosexual',
    'homosexual_gay_or_lesbian', 'bisexual', 'other_sexual_orientation',
    'christian', 'jewish', 'muslim', 'hindu', 'buddhist', 'atheist',
    'other_religion', 'black', 'white', 'asian', 'latino',
    'other_race_or_ethnicity', 'physical_disability',
    'intellectual_or_learning_disability', 'psychiatric_or_mental_illness',
    'other_disability'
]

COVERT_LABELS = [
    'explicitly_offensive', 'implicitly_offensive', 'not_sure_offensive',
    'not_offensive', 'covert_humor', 'covert_obfuscation',
    'covert_emoticons_emojis', 'covert_sarcasm', 'covert_microaggression',
    'covert_masked_harm', 'covert_political'
]

In [ ]:
df_covert = df_civil_full[~df_civil_full['covert_humor'].isnull()]
print(len(df_covert))
print(50529 - 48074 == 2455) # So by using a single filter we can get all of the covert rows
df_covert.head(3)

50529
True


,id,comment_text,created_date,publication_id,parent_id,article_id,rating,funny,wow,sad,...,covert_emoticons_emojis,covert_sarcasm,covert_microaggression,covert_masked_harm,covert_political,split,parent_text,spans,spans_split,contextual_toxicity
43,7097363,And let's not forget the original chemically p...,2016-10-18 02:30:46.198515+00,53,534102.0,148708,approved,0,0,0,...,0.0,0.0,0.2,0.0,0.0,test_private,NaN,NaN,NaN,NaN
55,7097375,Just because in the past people's mental healt...,2017-09-19 19:58:14.019533+00,54,5982112.0,379397,approved,0,0,0,...,0.2,0.0,0.2,0.0,0.0,test_private,NaN,NaN,NaN,NaN
79,7097399,States rights and white supremacy are not mutu...,2017-09-15 17:58:13.675003+00,13,NaN,378049,approved,0,0,0,...,0.0,0.0,0.0,0.0,0.0,test_private,NaN,NaN,NaN,NaN


In [ ]:
df_civil = df_civil_full[df_civil_full['covert_humor'].isnull()]
print(len(df_civil))
print(len(df_civil_full) - 1948985 == len(df_covert))  # Rest of civil comments
df_civil.head(3)

1948985
True


,id,comment_text,created_date,publication_id,parent_id,article_id,rating,funny,wow,sad,...,covert_emoticons_emojis,covert_sarcasm,covert_microaggression,covert_masked_harm,covert_political,split,parent_text,spans,spans_split,contextual_toxicity
0,7097320,[ Integrity means that you pay your debts.]\n\...,2017-09-13 20:12:01.484121+00,21,5945023.0,376974,approved,0,0,0,...,NaN,NaN,NaN,NaN,NaN,test_private,NaN,NaN,NaN,NaN
1,7097321,This is malfeasance by the Administrator and t...,2017-05-17 07:01:51.902566+00,55,NaN,335003,approved,0,0,0,...,NaN,NaN,NaN,NaN,NaN,test_private,NaN,NaN,NaN,NaN
2,7097322,@Rmiller101 - Spoken like a true elitist. But ...,2016-12-02 17:12:12.920957+00,54,649389.0,154126,approved,0,0,0,...,NaN,NaN,NaN,NaN,NaN,test_private,NaN,NaN,NaN,NaN


In [ ]:
df_civil_train = df_civil[df_civil['split'] ==  'train']
df_civil_val = df_civil[df_civil['split'] ==  'test_public']
df_civil_test = df_civil[df_civil['split'] ==  'test_private']
df_covert_train = df_covert[df_covert['split'] ==  'train']
df_covert_test = df_civil[df_civil['split'] ==  'test_private'] # test public is empty

In [ ]:
len(df_civil_train) + len(df_covert_train) # We have kept the original split removing covert

1804874

In [ ]:
import sweetviz as sv
my_report = sv.compare([df_civil_train, "Training Civil Comments Data"]
                       , [df_civil_val, "Validation Civil Comments Data"])
my_report.show_html("report_civil.html")

                                             |          | [  0%]   00:00 -> (? left)

Report report_civil.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [ ]:
my_report = sv.analyze([df_covert_train, "Training Covert Toxicity Data"])
my_report.show_html("report_covert.html")

                                             |          | [  0%]   00:00 -> (? left)

Report report_covert.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [ ]:
my_report = sv.compare([df_civil_train, "Training Civil Comments Data"]
                       ,[df_covert_train, "Training Covert Toxicity Data"])
my_report.show_html("report_civil_covert.html")

                                             |          | [  0%]   00:01 -> (? left)

Report report_civil_covert.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.
